In [0]:
import requests 
import json
from google.cloud import storage
from google.oauth2 import service_account
from pyspark.dbutils import DBUtils
import os

In [0]:
#Instancia DBUtils
dbutils = DBUtils()

# Chave de acesso ao GCP
key_path =  "/dbfs/key_account/chave_gcp_final.json"


# Configuração do ambiente GCP com a chave de acesso
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = key_path

# Efetuar o login com as credenciais
credentials = service_account.Credentials.from_service_account_file(key_path)
client = storage.Client(credentials=credentials)


In [0]:
#Folder
folder_path = "/temp"

# Tenta listar a pasta no DBFS ou cria se não existir
try:
    dbutils.fs.ls(folder_path)
    print(f"Pasta existe neste caminho: {folder_path}.")
except Exception as e:
    dbutils.fs.mkdirs(folder_path)
    print(f"Pasta criada em: {folder_path}.")

Pasta existe neste caminho: /temp.


In [0]:
# URL da API da AniList para buscar a lista de animes assistidos de um usuário
url = 'https://graphql.anilist.co'

query = '''
query ($username: String) {
  MediaListCollection(userName: $username, type: ANIME, status: COMPLETED) {
    lists {
      entries {
        media {
          title {
            romaji
            english
          }
          episodes
          coverImage {
            large
          }
          averageScore
          genres
          startDate {
            year
          }
          studios(isMain: true) {
            nodes {
              name
            }
          }
          description
          relations {
            edges {
              node {
                title {
                  romaji
                  english
                }
                type
                siteUrl
                coverImage {
                  large
                }
              }
              relationType
            }
          }
          popularity
          season
          seasonYear
          duration
          trending
          source
          favourites
        }
        completedAt {
          year
          month
          day
        }
        score
      }
    }
  }
}
'''


# Variáveis da consulta, incluindo o nome de usuário
variables = {
    'username': 'Gajs'
}

# Faz a requisição para a API da AniList
response = requests.post(url, json={'query': query, 'variables': variables})


In [0]:
 # Configuração do cliente do Google Cloud Storage
client = storage.Client()
    
# Nome do bucket no Google Cloud Storage
bucket_name = 'raw_bronze_files'
bucket = client.bucket(bucket_name)
    
# Nome do arquivo no GCS 
gcs_file_path = 'anime_data.json'
    
# Caminho completo do arquivo local no DBFS
local_file_path = f"{folder_path}/anime_data.json"
    
# Verificar se o arquivo local existe no DBFS
try:
    dbutils.fs.ls(local_file_path)
    file_exists = True
except Exception as e:
    file_exists = False
    
if file_exists:
    # Fazer o upload do arquivo para o GCS
    temp_local_path = "/dbfs" + local_file_path
    blob = bucket.blob(gcs_file_path)
    blob.upload_from_filename(temp_local_path)  
    
    print(f'Arquivo salvo em gs://{bucket_name}/{gcs_file_path}')
else:
    print(f'O arquivo {local_file_path} não existe.')   

Arquivo salvo em gs://raw_bronze_files/anime_data.json
